In [3]:
import numpy as np
import pandas as pd
import glob

# Pre-Processing

things to do: ICA filtering to remove artifacts, butterworth filter, ...

In [4]:
train_labels = pd.read_csv('Data/TrainLabels.csv')
submission = pd.read_csv('Data/SampleSubmission.csv')

There are 60 feedbacks for each session, AKA, 12 5 letter words. Each feedback/letter was either a right or wrong prediction from the user. Using the EEG data, we must train a model on the tendencies within the EEG data itself, whenever a feedback was presented.

In [5]:
train_labels.head(60)

,IdFeedBack,Prediction
0,S02_Sess01_FB001,1
1,S02_Sess01_FB002,1
2,S02_Sess01_FB003,0
3,S02_Sess01_FB004,0
4,S02_Sess01_FB005,1
5,S02_Sess01_FB006,0
6,S02_Sess01_FB007,1
7,S02_Sess01_FB008,1
8,S02_Sess01_FB009,0
9,S02_Sess01_FB010,1


Collecting all the names of the training files, and then running a loop through each file, it is imported as a DataFrame, and then turned into an array, where it is appended to the training/test set.

In [6]:
train_files = glob.glob('Data/train/Data*.csv')
test_files = glob.glob('Data/test/Data*.csv')
train_files[0:6]

['Data/train\\Data_S02_Sess01.csv',
 'Data/train\\Data_S02_Sess02.csv',
 'Data/train\\Data_S02_Sess03.csv',
 'Data/train\\Data_S02_Sess04.csv',
 'Data/train\\Data_S02_Sess05.csv',
 'Data/train\\Data_S06_Sess01.csv']

In [10]:
'''
Ingest Data by looping through files

Take only rows where FeedBackEvent == 1 (when the prediction was good) 
and then drop the column

Append values to list of arrays called temp

Input: files -> array of string of file names (Data_S*_Sess*.csv)
Output: temp -> final array of all appended values
'''
def extract_d(files):
    for i, f in enumerate(files):
        print(i,f)
        df = pd.read_csv(f)
        df = df[df.FeedBackEvent == 1]
        df = df.drop('FeedBackEvent', axis = 1)
        if i == 0:
            temp = np.array(df)
        else:
            temp = np.vstack((temp,np.array(df)))
            
    return temp

In [ ]:
train = extract_d(train_files)

In [11]:
test = extract_d(test_files)

0 Data/test\Data_S01_Sess01.csv
1 Data/test\Data_S01_Sess02.csv
2 Data/test\Data_S01_Sess03.csv
3 Data/test\Data_S01_Sess04.csv
4 Data/test\Data_S01_Sess05.csv
5 Data/test\Data_S03_Sess01.csv
6 Data/test\Data_S03_Sess02.csv
7 Data/test\Data_S03_Sess03.csv
8 Data/test\Data_S03_Sess04.csv
9 Data/test\Data_S03_Sess05.csv
10 Data/test\Data_S04_Sess01.csv
11 Data/test\Data_S04_Sess02.csv
12 Data/test\Data_S04_Sess03.csv
13 Data/test\Data_S04_Sess04.csv
14 Data/test\Data_S04_Sess05.csv
15 Data/test\Data_S05_Sess01.csv
16 Data/test\Data_S05_Sess02.csv
17 Data/test\Data_S05_Sess03.csv
18 Data/test\Data_S05_Sess04.csv
19 Data/test\Data_S05_Sess05.csv
20 Data/test\Data_S08_Sess01.csv
21 Data/test\Data_S08_Sess02.csv
22 Data/test\Data_S08_Sess03.csv
23 Data/test\Data_S08_Sess04.csv
24 Data/test\Data_S08_Sess05.csv
25 Data/test\Data_S09_Sess01.csv
26 Data/test\Data_S09_Sess02.csv
27 Data/test\Data_S09_Sess03.csv
28 Data/test\Data_S09_Sess04.csv
29 Data/test\Data_S09_Sess05.csv
30 Data/test\Data_S1

Output new arrays into 2 csvs for model use

In [14]:
cols = ['Time', 'Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3',
       'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz',
       'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
       'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
       'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7',
       'POz', 'P08', 'O1', 'O2', 'EOG']
test_df = pd.DataFrame(test, columns = cols)
test_df.to_csv('Data/test_df.csv')

In [263]:
cols = ['Time', 'Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3',
       'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz',
       'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
       'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
       'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7',
       'POz', 'P08', 'O1', 'O2', 'EOG']
train_df = pd.DataFrame(train, columns = cols)
train_df.to_csv('Data/train_df.csv')

# Visualizations